# Deploy to an online endpoint

To consume a model from an application, you can deploy the model to an online endpoint. You'll create an MLflow model from local files and test the endpoint.

## Before you start

You'll need the latest version of the  **azureml-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [1]:
!pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.14.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: c:\users\sirip\miniconda3\envs\myenv\lib\site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 


## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml import command
import os
from dotenv import load_dotenv


load_dotenv()
subscription_id = os.getenv('SUBSCRIPTION_ID')
resource_group = os.getenv('RESOURCE_GROUP')
workspace = os.getenv('WORKSPACE')
storage_account_name = os.getenv('STORAGE_ACCOUNT_NAME')
storage_account_key = os.getenv('STORAGE_ACCOUNT_KEY')


ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

ml_client

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x000002554D4A9D20>,
         subscription_id=73a3b9d4-4ac9-4810-8752-0f69cc4ac4a3,
         resource_group_name=DS_Team,
         workspace_name=mlw-example)

## Define and create an endpoint

Ultimately, the goal is to deploy a model to an endpoint. Therefore, you first need to create an endpoint. The endpoint will be a HTTPS endpoint that an application can call to receive predictions from the model. An application can consume an endpoint by using its URI, and authenticating with a key or token.

Run the following cell to define the endpoint. Note that the name of the endpoint has to be unique. You'll use the `datetime` function to generate a unique name.

In [2]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for MLflow diabetes model",
    auth_mode="key",
)

Next, you'll create the endpoint by running the following cell. This may take several minutes. While your endpoint is being created, you can read about [what are Azure Machine Learning endpoints](https://learn.microsoft.com/azure/machine-learning/concept-endpoints).

In [3]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-06181056234573.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-06181056234573.eastus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-06181056234573', 'description': 'Online endpoint for MLflow diabetes model', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/73a3b9d4-4ac9-4810-8752-0f69cc4ac4a3/resourcegroups/ds_team/providers/microsoft.machinelearningservices/workspaces/mlw-example/onlineendpoints/endpoint-06181056234573', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/73a3b9d4-4ac9-4810-8752-0f69cc4ac4a3/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oeidp:444254f6-1654-424b-9972-8f40ab1af838:e8b84c0c-23cb-498a-8153-c46c0f099abf?api-version=2022-02-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/73a3b9d4-4ac9-4810-8752-0f69cc4ac4a3/resourc

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the endpoint is created successfully before continuing! A green notification should appear in the studio. </p>

## Configure the deployment

You can deploy multiple models to an endpoint. This is mostly useful when you want to update the deployed model while keeping the current model in production. You'll need to configure the deployment to specify which model needs to be deployed to an endpoint. In the following cell, you'll refer to the model trained and stored in the local `model` folder (stored in the same folder as this notebook). Note that since you're working with an MLflow model, you don't need to specify the environment or scoring script.

You'll also specify the infrastructure needed for the model to be deployed.

In [4]:
from azure.ai.ml.entities import Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

# create a blue deployment
model = Model(
    path="./model",
    type=AssetTypes.MLFLOW_MODEL,
    description="my sample mlflow model",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

## Create the deployment

Finally, you can actually deploy the model to the endpoint by running the following cell:

In [5]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint endpoint-06181056234573 exists
Uploading model (0.0 MBs): 100%|##########| 2148/2148 [00:00<00:00, 3227.92it/s]




.............................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpoint-06181056234573', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/73a3b9d4-4ac9-4810-8752-0f69cc4ac4a3/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/odidp:444254f6-1654-424b-9972-8f40ab1af838:161f08fc-7c5c-4028-96d6-07263059365e?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/73a3b9d4-4ac9-4810-8752-0f69cc4ac4a3/resourceGroups/DS_Team/providers/Microsoft.MachineLearningServices/workspaces/mlw-example/onlineEndpoints/endpoint-06181056234573/deployments/blue', 'Resource__source_path': '', 'base_path': 'C:\\Users\\sirip\\Documents\\azure-machine-learning\\azure-ml-labs\\Labs\\11', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x00000255554C2

The deployment of the model may take 10-15 minutes. While waiting for the model to be deployed, you can learn more about [managed endpoints in this video](https://www.youtube.com/watch?v=SxFGw_OBxNM&ab_channel=MicrosoftDeveloper).

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio.</p>

Since you only have one model deployed to the endpoint, you want this deployment to take 100% of the traffic. If you deploy multiple models to the endpoint, you could use the same approach to distribute traffic across the deployed models.

In [6]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-06181056234573.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-06181056234573.eastus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-06181056234573', 'description': 'Online endpoint for MLflow diabetes model', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/73a3b9d4-4ac9-4810-8752-0f69cc4ac4a3/resourcegroups/ds_team/providers/microsoft.machinelearningservices/workspaces/mlw-example/onlineendpoints/endpoint-06181056234573', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/73a3b9d4-4ac9-4810-8752-0f69cc4ac4a3/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oeidp:444254f6-1654-424b-9972-8f40ab1af838:69d9194f-3408-4ac3-b65e-ac5259b10cb5?api-version=2022-02-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/73a3b9d4-4ac9-4810-8752-0f69cc4ac4a3/resourc

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the blue deployment is configured before continuing! A green notification should appear in the studio. </p> 

## Test the deployment

Let's test the deployed model by invoking the endpoint. A JSON file with sample data is used as input. The trained model predicts whether a patient has diabetes or not, based on medical data like age, BMI, and the number of pregnancies. A `[0]` indicates a patient doesn't have diabetes. A `[1]` means a patient does have diabetes.

In [7]:
# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample-data.json",
)

if response[1]=='1':
    print("Diabetic")
else:
    print ("Not diabetic")

Diabetic


Optionally, you can change the values in the `sample-data.json` file to try and get a different prediction.

## List endpoints

Although you can view all endpoints in the Studio, you can also list all endpoints using the SDK:

In [8]:
endpoints = ml_client.online_endpoints.list()
for endp in endpoints:
    print(endp.name)

endpoint-06181056234573


## Get endpoint details

If you want more information about a specific endpoint, you can explore the details using the SDK too.

In [9]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'blue': 100}
https://endpoint-06181056234573.eastus.inference.ml.azure.com/score


## Delete the endpoint and deployment

As an endpoint is always available, it can't be paused to save costs. To avoid unnecessary costs, delete the endpoint.

In [10]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)